In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

In [2]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import pickle
import gc
import json
import h5py

from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline


# == recnn ==
import sys
sys.path.append("../../")
import recnn

cuda = torch.device('cuda')
# ---
frame_size = 10
batch_size = 25
# --- 

tqdm.pandas()

In [3]:
# https://drive.google.com/open?id=1kTyu05ZmtP2MA33J5hWdX8OyUYEDW4iI
movie_embeddings_key_dict = pickle.load(open('../../data/embeddings/ml20_pca128.pkl', 'rb'))
movies_embeddings_tensor, key_to_id, id_to_key = recnn.data.make_items_tensor(movie_embeddings_key_dict)
# download ml20m dataset yourself
ratings = pd.read_csv('../../data/ml-20m/ratings.csv')
user_dict, users = recnn.data.prepare_dataset(ratings, key_to_id, frame_size)

del ratings
gc.collect()
clear_output(True)
clear_output(True)
print('Done!')

Done!


In [6]:
# === ddpg settings ===

params = {
    'gamma'      : 0.99,
    'min_value'  : -10,
    'max_value'  : 10,
    'policy_step': 10,
    'soft_tau'   : 0.001,
    
    'policy_lr'  : 1e-5,
    'value_lr'   : 1e-5,
    'actor_weight_init': 54e-2,
    'critic_weight_init': 6e-1,
}

env = {'original_state_dim': 129, 'state_dim': 256, 'action_dim': 128, 'hidden_dim': 256}

n_epochs = 100
batch_size = 25

epoch_bar = tqdm(total=n_epochs)

# Dataset preparation
train_users = recnn.data.sort_users_itemwise(user_dict, users[:-5000])[2:]
test_users = recnn.data.sort_users_itemwise(user_dict, users[-5000:])

def prepare_batch_wrapper(batch):
    batch = recnn.data.padder(batch)
    batch = recnn.data.prepare_batch_dynamic_size(batch, movies_embeddings_tensor)
    return batch
    
train_user_dataset = recnn.data.UserDataset(train_users, user_dict)
test_user_dataset = recnn.data.UserDataset(test_users, user_dict)
train_dataloader = DataLoader(train_user_dataset, batch_size=batch_size,
                              num_workers=4,collate_fn=prepare_batch_wrapper)
test_dataloader = DataLoader(test_user_dataset, batch_size=batch_size,
                             num_workers=4,collate_fn=prepare_batch_wrapper)

dataloader = {'test': test_dataloader, 'train': train_dataloader}
             
# writer and debugger setup 
writer = SummaryWriter(log_dir='../../runs')
debugger = recnn.Debugger({}, run_tests, writer)


DDPG = recnn.algo.DDPG(env, params, dataloader, debugger, 'lstm', torch.device('cpu'))

# === end ===

NameError: name 'run_tests' is not defined